In [1]:
# # Add this cell BEFORE starting servers
# import subprocess
# import os

# # Kill any existing processes on our ports
# for port in [10020, 10021, 10022, 8001]:
#     try:
#         # Find and kill process using the port (macOS/Linux)
#         result = subprocess.run(
#             f"lsof -ti:{port} | xargs kill -9",
#             shell=True,
#             capture_output=True
#         )
#     except:
#         pass

# print("Cleared ports 10020, 10021, 10022, 8001")

In [2]:
!pip show a2a-sdk google-adk | grep -E "^(Name|Version)"


Name: a2a-sdk
Version: 0.3.20
Name: google-adk
Version: 1.20.0


In [3]:
import sys

from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver


class PatchedClientModule:
    def __init__(self, real_module) -> None:
        for attr in dir(real_module):
            if not attr.startswith('_'):
                setattr(self, attr, getattr(real_module, attr))
        self.A2ACardResolver = A2ACardResolver


patched_module = PatchedClientModule(real_client_module)
sys.modules['a2a.client.client'] = patched_module  # type: ignore


In [4]:
import asyncio
import logging
import os
import sys
import threading
import time

from typing import Any
import uuid

import httpx
import nest_asyncio
import uvicorn

from a2a.client import ClientConfig, ClientFactory, create_text_message_object
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
    TransportProtocol,
    Message, Part, TextPart, Role
)
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH
from dotenv import load_dotenv
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.agents import Agent, SequentialAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent

from google.adk.tools import google_search
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset
from google.adk.tools.mcp_tool.mcp_session_manager import SseServerParams

In [5]:
MODEL_NAME = "gemini-2.5-flash"
MCP_SERVER_URL = "http://127.0.0.1:8001/sse"

CUSTOMER_DATA_AGENT_URL = "http://127.0.0.1:10020"
SUPPORT_AGENT_URL = "http://127.0.0.1:10021"
ROUTER_AGENT_URL = "http://127.0.0.1:10022"


In [6]:
import logging
logging.basicConfig(
    level=logging.WARNING,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("google.adk").setLevel(logging.WARNING)
logging.getLogger("a2a").setLevel(logging.WARNING)

logger = logging.getLogger("customer_service_system")

In [7]:
# Import API key
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

# Defining Agents

1. Router Agent
2. Customer Data Agent
3. Support Agent

In [8]:

# Support Agent
def build_support_agent() -> Agent:
    mcp_toolset = MCPToolset(
        connection_params=SseServerParams(
            url=MCP_SERVER_URL,
            timeout=300.0,
            sse_read_timeout=300.0
        )
    )
    instruction = """
    You are the Support Agent. You handle customer support requests.
    
    Available MCP Tools:
    - create_ticket(customer_id, issue, priority): Create a support ticket
    - get_customer_history(customer_id): Get ticket history
    
    Responsibilities:
    - Analyze customer issues and determine priority (low/medium/high)
    - Create tickets for issues that need tracking
    - Provide helpful responses to customer queries
    - Escalate urgent issues (billing disputes, account access) as high priority
    
    When you receive customer context from another agent, use it to personalize your response.
    
    Priority Guidelines:
    - HIGH: Billing issues, account access problems, security concerns
    - MEDIUM: Feature questions, general complaints, update requests
    - LOW: General inquiries, feature requests
    
    return Agent(
        model=MODEL_NAME,
        name="router_agent",
        instruction=instruction,
        tools=[mcp_toolset]
    )
    """
    return Agent(
        model=MODEL_NAME,
        name="support_agent",
        instruction=instruction
    )

# Customer data
def build_customer_data_agent() -> Agent:
    mcp_toolset = MCPToolset(
        connection_params=SseServerParams(
            url=MCP_SERVER_URL,
            timeout=300.0,
            sse_read_timeout=300.0
        )
    )
    instruction = """
    You are the Customer Data Agent. You have exclusive access to the customer database.
    
    Available MCP Tools:
    - get_customer(customer_id): Get a single customer by ID
    - list_customers(status, limit): List customers by status ('active' or 'disabled')
    - update_customer(customer_id, data): Update customer fields
    - create_ticket(customer_id, issue, priority): Create a support ticket
    - get_customer_history(customer_id): Get all tickets for a customer
    
    When asked for customer information:
    1. Use the appropriate MCP tool
    2. Return the data in a clear, structured format
    3. Include all relevant fields
    
    Always respond with factual data from the database. Never make up customer information.
    """
    return Agent(
        model=MODEL_NAME,
        name="customer_data_agent",
        instruction=instruction,
        tools=[mcp_toolset],
    )

# Router
def build_router_agent(remote_customer_data: RemoteA2aAgent, remote_support: RemoteA2aAgent) -> Agent:
    instruction = """
    You are the Support Agent. You handle customer support requests.
    
    Available MCP Tools:
    - create_ticket(customer_id, issue, priority): Create a support ticket
    - get_customer_history(customer_id): Get all tickets for a customer
    
    IMPORTANT: You can call get_customer_history multiple times for different customers.
    
    Responsibilities:
    - Analyze customer issues and determine priority (low/medium/high)
    - Create tickets for issues that need tracking
    - Provide helpful responses to customer queries
    - Escalate urgent issues (billing disputes, account access) as high priority
    
    Priority Guidelines:
    - HIGH: Billing issues, account access problems, security concerns
    - MEDIUM: Feature questions, general complaints, update requests
    - LOW: General inquiries, feature requests
"""
    host = SequentialAgent(
        name="router_host_agent",
        sub_agents=[remote_customer_data, remote_support]
    )
    return host


# A2A Orchestration Loop

In [9]:
# Simplified Server Infrastructure using to_a2a
async def run_agent_server(agent: Any, agent_card: AgentCard, port: int) -> None:
    print(f"Starting server for {agent.name} on port {port}...")
    # to_a2a converts the agent to a Starlette app automatically
    app = to_a2a(agent, agent_card=agent_card, port=port)
    
    config = uvicorn.Config(
        app,

        port=port,
        log_level="warning",
        loop="none",
    )
    server = uvicorn.Server(config)
    await server.serve()

print("A2A server infrastructure defined (using to_a2a).")


A2A server infrastructure defined (using to_a2a).


In [10]:
print("\nAgentCard fields:")
for field_name, field_info in AgentCard.model_fields.items():
    required = "REQUIRED" if field_info.is_required() else "optional"
    print(f"  {field_name}: {required}")
from google.adk.a2a.utils.agent_to_a2a import to_a2a



AgentCard fields:
  additional_interfaces: optional
  capabilities: REQUIRED
  default_input_modes: REQUIRED
  default_output_modes: REQUIRED
  description: REQUIRED
  documentation_url: optional
  icon_url: optional
  name: REQUIRED
  preferred_transport: optional
  protocol_version: optional
  provider: optional
  security: optional
  security_schemes: optional
  signatures: optional
  skills: REQUIRED
  supports_authenticated_extended_card: optional
  url: REQUIRED
  version: REQUIRED


In [11]:
# Customer Data Agent Card
customer_data_agent_card = AgentCard(
    name="Customer Data Agent",
    url=CUSTOMER_DATA_AGENT_URL,
    description="Accesses customer and ticket data via MCP tools. The only agent with direct database access.",
    version="1.0",
    capabilities=AgentCapabilities(streaming=True),
    defaultInputModes=["text/plain"],
    defaultOutputModes=["application/json"],
    skills=[
        AgentSkill(
            id="customer_data_operations",
            name="Customer Data Operations",
            description="Retrieve, update, and manage customer records and ticket history",
            tags=["customer", "database", "mcp", "tickets"],
            examples=[
                "Get customer information for ID 5",
                "List all active customers",
                "Update customer email",
                "Show ticket history for customer 3",
            ],
        )
    ],
)

# Support Agent Card
support_agent_card = AgentCard(
    name="Support Agent",
    url=SUPPORT_AGENT_URL,
    description="Handles customer support conversations, assesses priorities, and creates tickets",
    version="1.0",
    capabilities=AgentCapabilities(streaming=True),
    defaultInputModes=["text/plain"],
    defaultOutputModes=["application/json"],
    skills=[
        AgentSkill(
            id="support_handling",
            name="Customer Support Handling",
            description="Handle support issues, assess priority, create tickets, provide resolutions",
            tags=["support", "billing", "escalation", "tickets"],
            examples=[
                "I need help with my account",
                "I was charged twice, please refund",
                "How do I upgrade my subscription?",
            ],
        )
    ],
)

# Router Agent Card
router_agent_card = AgentCard(
    name="Router Agent",
    url=ROUTER_AGENT_URL,
    description="Orchestrates customer queries by coordinating between Customer Data and Support agents",
    version="1.0",
    capabilities=AgentCapabilities(streaming=True),
    defaultInputModes=["text/plain"],
    defaultOutputModes=["application/json"],
    skills=[
        AgentSkill(
            id="query_orchestration",
            name="Customer Service Orchestration",
            description="Routes queries to appropriate agents and coordinates multi-agent responses",
            tags=["routing", "orchestration", "multi-agent"],
            examples=[
                "I'm customer 5 and need help upgrading my account",
                "I've been charged twice, please refund immediately!",
                "Update my email and show my ticket history",
            ],
        )
    ],
)

print("Agent Cards defined:")
print(f"  - {customer_data_agent_card.name}")
print(f"  - {support_agent_card.name}")
print(f"  - {router_agent_card.name}")

Agent Cards defined:
  - Customer Data Agent
  - Support Agent
  - Router Agent


In [12]:
# Build the actual ADK agents
customer_data_agent = build_customer_data_agent()
support_agent = build_support_agent()

print("Built specialist agents:")
print(f"  - {customer_data_agent.name}")
print(f"  - {support_agent.name}")

Built specialist agents:
  - customer_data_agent
  - support_agent


/var/folders/qf/l8sfrcpd6vq2564v1_qt31fm0000gn/T/ipykernel_63428/1112252722.py:45: DeprecationWarning: MCPToolset class is deprecated, use `McpToolset` instead.
  mcp_toolset = MCPToolset(
/var/folders/qf/l8sfrcpd6vq2564v1_qt31fm0000gn/T/ipykernel_63428/1112252722.py:3: DeprecationWarning: MCPToolset class is deprecated, use `McpToolset` instead.
  mcp_toolset = MCPToolset(


In [13]:
# Create remote agents
remote_customer_data_agent = RemoteA2aAgent(
    name="customer_data_remote",
    description="Remote A2A wrapper for Customer Data Agent - accesses database via MCP",
    agent_card=f"{CUSTOMER_DATA_AGENT_URL}{AGENT_CARD_WELL_KNOWN_PATH}",
)

remote_support_agent = RemoteA2aAgent(
    name="support_remote",
    description="Remote A2A wrapper for Support Agent - handles customer issues",
    agent_card=f"{SUPPORT_AGENT_URL}{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("Created remote agent wrappers:")
print(f"  - {remote_customer_data_agent.name} -> {CUSTOMER_DATA_AGENT_URL}")
print(f"  - {remote_support_agent.name} -> {SUPPORT_AGENT_URL}")

Created remote agent wrappers:
  - customer_data_remote -> http://127.0.0.1:10020
  - support_remote -> http://127.0.0.1:10021


/var/folders/qf/l8sfrcpd6vq2564v1_qt31fm0000gn/T/ipykernel_63428/176174927.py:2: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  remote_customer_data_agent = RemoteA2aAgent(
/var/folders/qf/l8sfrcpd6vq2564v1_qt31fm0000gn/T/ipykernel_63428/176174927.py:8: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  remote_support_agent = RemoteA2aAgent(


In [14]:
# Router uses the remote wrappers to coordinate
router_agent = build_router_agent(
    remote_customer_data=remote_customer_data_agent,
    remote_support=remote_support_agent,
)

print(f"Built router agent: {router_agent.name}")
print(f"  Sub-agents: {[a.name for a in router_agent.sub_agents]}")

Built router agent: router_host_agent
  Sub-agents: ['customer_data_remote', 'support_remote']


In [15]:
# Start A2A Server
async def start_all_servers() -> None:
    tasks = [
        asyncio.create_task(
            run_agent_server(customer_data_agent, customer_data_agent_card, 10020)
        ),
        asyncio.create_task(
            run_agent_server(support_agent, support_agent_card, 10021)
        ),
        asyncio.create_task(
            run_agent_server(router_agent, router_agent_card, 10022)
        ),
    ]
    
    # Wait a moment for servers to start
    await asyncio.sleep(2.0)
    
    logger.info("A2A servers started:")
    logger.info(f"  Customer Data Agent: {CUSTOMER_DATA_AGENT_URL}")
    logger.info(f"  Support Agent:       {SUPPORT_AGENT_URL}")
    logger.info(f"  Router Agent:        {ROUTER_AGENT_URL}")
    
    try:
        await asyncio.gather(*tasks)
    except asyncio.CancelledError:
        logger.info("Server tasks cancelled")


def run_servers_in_background() -> None:
    """
    Run servers in a background thread (for notebook compatibility).
    """
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    try:
        loop.run_until_complete(start_all_servers())
    except Exception as e:
        print(f"Server error: {e}")

print("Server startup functions defined.")

Server startup functions defined.


In [16]:
from a2a.types import Message, Part, TextPart, Role
from google.adk.a2a.utils.agent_to_a2a import to_a2a


In [17]:
# A2A CLIENT - For sending queries to agents
import uuid
from uuid import uuid4
from typing import Dict, List, Any

from a2a.types import SendMessageRequest, MessageSendParams
from a2a.client import A2AClient as A2ABaseClient
from a2a.client.card_resolver import A2ACardResolver

class A2AClient:
    """
    Simple client for calling A2A servers.
    Used to send test queries to the Router Agent.
    """
    
    def __init__(self, default_timeout: float = 240.0):
        self._agent_card_cache: Dict[str, Any] = {}
        self.default_timeout = default_timeout
    
    async def send_query(self, agent_url: str, message: str) -> str:
        """
        Send a query to an A2A agent and get the response.
        
        Args:
            agent_url: Base URL of the agent (e.g., http://127.0.0.1:10022)
            message: The query to send
            
        Returns:
            The agent's response text
        """
        timeout_config = httpx.Timeout(
            timeout=self.default_timeout,
            connect=10.0,
            read=self.default_timeout,
            write=10.0,
            pool=5.0,
        )
        
        async with httpx.AsyncClient(timeout=timeout_config) as client:
            jsonrpc_request = {
                "jsonrpc": "2.0",
                "id": uuid4().hex,
                "method": "message/send",
                "params": {
                    "message": {
                        "role": "user",
                        "parts": [
                            {"kind": "text", "text": message}
                        ],
                        "messageId": uuid4().hex  # camelCase - THIS IS THE KEY!
                    }
                }
            }

            

            # Send and get response (non-streaming)
            response = await client.post(
                agent_url,
                json=jsonrpc_request,
                headers={"Content-Type": "application/json"}
            )
            
            result = response.json()
            
            if "error" in result:
                return f"Error: {result['error']}"
            
            # Extract text from successful response
            if "result" in result:
                res = result["result"]
                
                # Handle Message response
                if "parts" in res:
                    for part in res["parts"]:
                        if "text" in part:
                            return part["text"]
                
                # Handle Task response with artifacts
                if "artifacts" in res:
                    for artifact in res["artifacts"]:
                        if "parts" in artifact:
                            for part in artifact["parts"]:
                                if "text" in part:
                                    return part["text"]
                
                # Handle Task response - return status info
                if "status" in res:
                    status = res["status"]
                    if "message" in status:
                        return status["message"]
                    return f"Task status: {status.get('state', 'unknown')}"
                
                return json.dumps(res, indent=2)
            
            return str(result)

print("A2A Client defined (using raw HTTP).")
from google.adk.a2a.utils.agent_to_a2a import to_a2a


A2A Client defined (using raw HTTP).


In [18]:
async def run_test_scenarios() -> None:
    client = A2AClient()
    
    # Scenario 1
    print("\n" + "="*70)
    print("SCENARIO 1: Simple Query")
    print("="*70)
    response1 = await client.send_query(ROUTER_AGENT_URL, "Get customer information for ID 5")
    print(f"\nRESPONSE:\n{response1}")
    
    print("\n⏳ Waiting 15 seconds...")
    await asyncio.sleep(15)
    
    # Scenario 2
    print("\n" + "="*70)
    print("SCENARIO 2: Coordinated Query")
    print("="*70)
    response2 = await client.send_query(ROUTER_AGENT_URL, "I'm customer 5 and need help upgrading my account to premium tier.")
    print(f"\nRESPONSE:\n{response2}")
    
    print("\n⏳ Waiting 15 seconds...")
    await asyncio.sleep(15)
    
    # Scenario 3
    print("\n" + "="*70)
    print("SCENARIO 3: Complex Query")
    print("="*70)
    response3 = await client.send_query(ROUTER_AGENT_URL, "Show me all active customers who have open tickets. Summarize them grouped by customer with ticket priorities.")
    print(f"\nRESPONSE:\n{response3}")
    
    print("\n⏳ Waiting 15 seconds...")
    await asyncio.sleep(15)
    
    # Scenario 4
    print("\n" + "="*70)
    print("SCENARIO 4: Escalation")
    print("="*70)
    response4 = await client.send_query(ROUTER_AGENT_URL, "I'm customer 1. I've been charged twice, please refund immediately! I am very upset.")
    print(f"\nRESPONSE:\n{response4}")
    
    print("\n⏳ Waiting 15 seconds...")
    await asyncio.sleep(15)
    
    # Scenario 5
    print("\n" + "="*70)
    print("SCENARIO 5: Multi-Intent")
    print("="*70)
    response5 = await client.send_query(ROUTER_AGENT_URL, "I'm customer 2. Update my email to new@email.com and then show my ticket history.")
    print(f"\nRESPONSE:\n{response5}")
    
    print("\n" + "="*70)
    print("ALL TEST SCENARIOS COMPLETED")
    print("="*70)

# Testing

In [19]:
import threading
import time
    
print("\n[1/3] Starting A2A servers in background...")
server_thread = threading.Thread(
    target=run_servers_in_background,
    daemon=True,
)
server_thread.start()

print("[2/3] Waiting for servers to initialize...")
time.sleep(5.0)

    
print("[3/3] Running test scenarios...\n")

/var/folders/qf/l8sfrcpd6vq2564v1_qt31fm0000gn/T/ipykernel_63428/1105539043.py:5: UserWarning: [EXPERIMENTAL] to_a2a: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  app = to_a2a(agent, agent_card=agent_card, port=port)
/Users/brunamedeiros/Documents/GitHub/AdvancedGenAI-HW5-Option-A/.venv/lib/python3.11/site-packages/google/adk/a2a/utils/agent_to_a2a.py:137: UserWarning: [EXPERIMENTAL] A2aAgentExecutor: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.



[1/3] Starting A2A servers in background...
[2/3] Waiting for servers to initialize...
Starting server for customer_data_agent on port 10020...
Starting server for support_agent on port 10021...
Starting server for router_host_agent on port 10022...
[3/3] Running test scenarios...



In [21]:
async def run_test_scenarios() -> None:
    client = A2AClient()
    
    # =========================================================================
    # Scenario 1: Simple Query
    # =========================================================================
    print("\n" + "="*70)
    print("SCENARIO 1: Simple Query")
    print("Query: 'Get customer information for ID 5'")
    print("Expected: Single agent, straightforward MCP call")
    print("="*70)
    
    response1 = await client.send_query(
        ROUTER_AGENT_URL,
        "Get customer information for ID 5"
    )
    print(f"\nRESPONSE:\n{response1}")

await run_test_scenarios()

2025-12-04 22:39:10,818 - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False



SCENARIO 1: Simple Query
Query: 'Get customer information for ID 5'
Expected: Single agent, straightforward MCP call


2025-12-04 22:39:12,392 - INFO - Response received from the model.
2025-12-04 22:39:12,421 - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-04 22:39:13,273 - INFO - Response received from the model.
2025-12-04 22:39:13,302 - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-04 22:39:14,404 - INFO - Response received from the model.



RESPONSE:
Here is the information for customer ID 5:

*   **ID**: 5
*   **Name**: Eve Martinez
*   **Email**: eve@email.com
*   **Phone**: 555-0105
*   **Status**: active
*   **Created At**: 2025-12-04 19:55:31
*   **Updated At**: 2025-12-04 19:55:31


In [22]:
async def run_test_scenarios() -> None:
    client = A2AClient()
    # =========================================================================
    # Scenario 2: Coordinated Query
    # =========================================================================
    print("\n" + "="*70)
    print("SCENARIO 2: Coordinated Query")
    print("Query: 'I'm customer 5 and need help upgrading my account'")
    print("="*70)
    
    response2 = await client.send_query(
        ROUTER_AGENT_URL,
        "I'm customer 5 and need help upgrading my account to premium tier."
    )
    print(f"\nRESPONSE:\n{response2}")
await run_test_scenarios()
    

2025-12-04 22:39:14,491 - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False



SCENARIO 2: Coordinated Query
Query: 'I'm customer 5 and need help upgrading my account'


2025-12-04 22:39:16,228 - INFO - Response received from the model.
2025-12-04 22:39:16,304 - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-04 22:39:18,165 - INFO - Response received from the model.
2025-12-04 22:39:18,216 - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-04 22:39:24,000 - INFO - Response received from the model.
2025-12-04 22:39:24,019 - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-04 22:39:26,347 - INFO - Response received from the model.



RESPONSE:
Thank you for letting me know! I see that a support ticket (ID: 33) has already been created for you to upgrade your account to the premium tier. A representative will be in touch with you shortly to assist with the upgrade.


In [23]:
async def run_test_scenarios() -> None:
    client = A2AClient()
    # =========================================================================
    # Scenario 3: Complex Query
    # =========================================================================
    print("\n" + "="*70)
    print("SCENARIO 3: Complex Query")
    print("Query: 'Show me all active customers who have open tickets'")
    print("="*70)
    
    response3 = await client.send_query(
        ROUTER_AGENT_URL,
        "Show me all active customers who have open tickets. Summarize them grouped by customer with ticket priorities."
    )
    print(f"\nRESPONSE:\n{response3}")
await run_test_scenarios()

2025-12-04 22:39:26,427 - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False



SCENARIO 3: Complex Query
Query: 'Show me all active customers who have open tickets'


2025-12-04 22:39:29,478 - INFO - Response received from the model.
2025-12-04 22:39:29,541 - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-04 22:39:32,706 - INFO - Response received from the model.
2025-12-04 22:39:32,764 - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-04 22:39:38,750 - INFO - Response received from the model.
2025-12-04 22:39:38,779 - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-04 22:39:40,717 - INFO - Response received from the model.



RESPONSE:
Here is a summary of all active customers with open tickets, grouped by customer and their ticket priorities:

**Alice Johnson (ID: 1)**
*   **Open Ticket Priorities**: 6 High, 1 Medium

**Charlie Brown (ID: 3)**
*   **Open Ticket Priorities**: 2 High

**Eve Martinez (ID: 5)**
*   **Open Ticket Priorities**: 2 High, 4 Medium

**Frank Wilson (ID: 6)**
*   **Open Ticket Priorities**: 1 High

**Grace Lee (ID: 7)**
*   **Open Ticket Priorities**: 1 Medium

**Ivy Chen (ID: 9)**
*   **Open Ticket Priorities**: 1 Low

**Jack Thompson (ID: 10)**
*   **Open Ticket Priorities**: 1 Low

**Karen White (ID: 11)**
*   **Open Ticket Priorities**: 2 High

**Maria Rodriguez (ID: 13)**
*   **Open Ticket Priorities**: 1 Medium

**Nathan Kim (ID: 14)**
*   **Open Ticket Priorities**: 1 Low

**Olivia Taylor (ID: 15)**
*   **Open Ticket Priorities**: 1 High


In [24]:
async def run_test_scenarios() -> None:
    client = A2AClient()
    # =========================================================================
    # Scenario 4: Escalation
    # =========================================================================
    print("\n" + "="*70)
    print("SCENARIO 4: Escalation")
    print("Query: 'I've been charged twice, please refund immediately!'")
    print("="*70)
    
    response4 = await client.send_query(
        ROUTER_AGENT_URL,
        "I'm customer 1. I've been charged twice, please refund immediately! I am very upset."
    )
    print(f"\nRESPONSE:\n{response4}")
await run_test_scenarios()

2025-12-04 22:39:40,762 - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False



SCENARIO 4: Escalation
Query: 'I've been charged twice, please refund immediately!'


2025-12-04 22:39:41,930 - INFO - Response received from the model.
2025-12-04 22:39:41,962 - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-04 22:39:43,902 - INFO - Response received from the model.
2025-12-04 22:39:43,923 - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-04 22:39:46,032 - INFO - Response received from the model.



RESPONSE:
Hello! I understand how frustrating it must be to be charged twice, and I apologize for the inconvenience this has caused.

I see that a high-priority ticket (ID 34) has already been created for you regarding the duplicate charge and immediate refund request. Our team is actively investigating this issue and will work to resolve it promptly. You will be contacted as soon as there is an update.


In [25]:
async def run_test_scenarios() -> None:
    client = A2AClient()
    # =========================================================================
    # Scenario 5: Multi-Intent
    # =========================================================================
    print("\n" + "="*70)
    print("SCENARIO 5: Multi-Intent")
    print("Query: 'Update my email to new@email.com and show my ticket history'")
    print("="*70)
    
    response5 = await client.send_query(
        ROUTER_AGENT_URL,
        "I'm customer 2. Update my email to new@email.com and then show my ticket history."
    )
    print(f"\nRESPONSE:\n{response5}")
await run_test_scenarios()

2025-12-04 22:39:46,081 - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False



SCENARIO 5: Multi-Intent
Query: 'Update my email to new@email.com and show my ticket history'


2025-12-04 22:39:46,907 - INFO - Response received from the model.
2025-12-04 22:39:46,939 - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-04 22:39:47,447 - INFO - Response received from the model.
2025-12-04 22:39:47,491 - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-04 22:39:48,605 - INFO - Response received from the model.
2025-12-04 22:39:48,635 - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-04 22:39:50,471 - INFO - Response received from the model.



RESPONSE:
Your email has been updated to new@email.com. Here is your ticket history:

*   **Ticket ID:** 4
    *   **Customer ID:** 2
    *   **Issue:** Need help upgrading subscription
    *   **Status:** in_progress
    *   **Priority:** low
    *   **Created At:** 2025-12-04 19:55:31

*   **Ticket ID:** 5
    *   **Customer ID:** 2
    *   **Issue:** Billing cycle question
    *   **Status:** resolved
    *   **Priority:** low
    *   **Created At:** 2025-12-04 19:55:31


2025-12-04 22:52:48,173 - ERROR - Error in sse_reader
Traceback (most recent call last):
  File "/Users/brunamedeiros/Documents/GitHub/AdvancedGenAI-HW5-Option-A/.venv/lib/python3.11/site-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/Users/brunamedeiros/Documents/GitHub/AdvancedGenAI-HW5-Option-A/.venv/lib/python3.11/site-packages/httpx/_transports/default.py", line 271, in __aiter__
    async for part in self._httpcore_stream:
  File "/Users/brunamedeiros/Documents/GitHub/AdvancedGenAI-HW5-Option-A/.venv/lib/python3.11/site-packages/httpcore/_async/connection_pool.py", line 407, in __aiter__
    raise exc from None
  File "/Users/brunamedeiros/Documents/GitHub/AdvancedGenAI-HW5-Option-A/.venv/lib/python3.11/site-packages/httpcore/_async/connection_pool.py", line 403, in __aiter__
    async for part in self._stream:
  File "/Users/brunamedeiros/Documents/GitHub/AdvancedGenAI-HW5-Option-A/.venv/lib/python3.11/site-packages/httpcore/_asyn

**Main challenges:** I had a few challenges during this assignment. 
- I first tried building the system with LangGraph, and I truly had a tough time. I thought the SDK system was indeed easier to set up. Nonetheless, it was still challenging. I struggled to workout the two different ways A2A could've been set up. Originally, I attempted to do manual set up, but later on I changed to `to_a2a()`.
- Additionally, I spent way too long debugging why my `AgentCards` weren't working. While the python module uses snake_case, the `AgentCard` parameters need parameters in camelCase.
- I would constantly getthis cryptic error: "Expected response header Content-Type to contain 'text/event-stream', got 'application/json'". My A2A client was trying to stream responses but the server was sending regular JSON. The agents were speaking different protocols. Had to make sure everyone was on the same page about how they're communicating.